In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!pip install torch_nightly -f https://download.pytorch.org/whl/nightly/cu92/torch_nightly.html
!pip install fastai

Looking in links: https://download.pytorch.org/whl/nightly/cu92/torch_nightly.html
    100% |████████████████████████████████| 576.2MB 27kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x6295a000 @  0x7f392dae12a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641
    100% |████████████████████████████████| 143kB 6.9MB/s 
    100% |████████████████████████████████| 23.3MB 1.3MB/s 
    100% |████████████████████████████████| 1.9MB 11.0MB/s 
    100% |████████████████████████████████| 61kB 23.4MB/s 
    100% |████████████████████████████████| 591.8MB 26kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x61b28000 @  0x7fe909c832a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x5

In [ ]:
# import libraries
import fastai
from fastai import *
from fastai.text import * 
import pandas as pd
import numpy as np
from functools import partial
import io
import os

In [ ]:
truth_data = pd.read_json("truth.jsonl", lines=True)
truth_data.head()
#testing data
Ttruth_data = pd.read_json("Ttruth.jsonl", lines=True)

In [ ]:
clickbait_data = pd.read_json("instances.jsonl", lines=True)
clickbait_data.head()
#test data
Tclickbait_data = pd.read_json("Tinstances.jsonl", lines=True)

In [ ]:
data = pd.merge(clickbait_data,truth_data, on = 'id')
data.head()
Tdata = pd.merge(Tclickbait_data,Ttruth_data, on = 'id')

In [ ]:
data.shape

(19538, 14)

In [ ]:
df = pd.DataFrame({'label':data.truthClass, 'text':data.postText})

In [ ]:
Tdf = pd.DataFrame({'label':Tdata.truthClass, 'text':Tdata.postText})

In [ ]:
df.head()

,label,text
0,no-clickbait,[UK’s response to modern slavery leaving victi...
1,clickbait,[this is good]
2,no-clickbait,"[The ""forgotten"" Trump roast: Relive his bruta..."
3,clickbait,[Meet the happiest #dog in the world!]
4,no-clickbait,[Tokyo's subway is shut down amid fears over a...


In [ ]:
# clean up 
# 2nd argument of replace must be str
def clean_up(label):
    label = label.replace("no-clickbait", "0")
    label = label.replace("clickbait", "1")
    return label

In [ ]:
# convert to series and clean up the labels column
df["label"] = df.iloc[:,0].apply(clean_up)

In [ ]:
l = []
for i in range(len(df)):
  l.append(int(df.iloc[:,0][i]))


In [ ]:
df["label"] = l

In [ ]:
df.head()

,label,text
0,0,[UK’s response to modern slavery leaving victi...
1,1,[this is good]
2,0,"[The ""forgotten"" Trump roast: Relive his bruta..."
3,1,[Meet the happiest #dog in the world!]
4,0,[Tokyo's subway is shut down amid fears over a...


In [ ]:
t = []
for i in range(len(df)):
  t.append(df.iloc[:,1][i][0])


In [ ]:
df["text"] = t

In [ ]:
df['label'].value_counts()

0    14777
1     4761
Name: label, dtype: int64

In [ ]:
Tdf['label'].value_counts()

no-clickbait    1697
clickbait        762
Name: label, dtype: int64

both datasets 1 and 2 are imbalanced since the number of non-clickbait tweets in datasets 1 and 2 are 2.1 and 3.1 times bigger than the number of clickbait tweets respectively.

In [ ]:
#change df_trn and df_val
Tdf["label"] = Tdf.iloc[:,0].apply(clean_up)
l = []
for i in range(len(Tdf)):
  l.append(int(Tdf.iloc[:,0][i]))
Tdf["label"] = l

t = []
for i in range(len(Tdf)):
  t.append(Tdf.iloc[:,1][i][0])
Tdf["text"] = t

In [ ]:
Tdf.head()

,label,text
0,0,Apple's iOS 9 'App thinning' feature will give...
1,0,RT @kenbrown12: Emerging market investors are ...
2,1,U.S. Soccer should start answering tough quest...
3,0,How theme parks like Disney World left the mid...
4,1,Could light bulbs hurt your health? One compan...


In [ ]:
df_trn, df_val = Tdf, df

In [ ]:
df_trn.shape, df_val.shape

((2459, 2), (19538, 2))

In [ ]:
# Language model data
data_lm = TextLMDataBunch.from_df(train_df = df_trn, valid_df = df_val, path = "")

# Classifier model data
data_clas = TextClasDataBunch.from_df(path = "", train_df = df_trn, valid_df = df_val, vocab=data_lm.train_ds.vocab, bs=32)

learn = language_model_learner(data_lm, pretrained_model=URLs.WT103_1, drop_mult=0.3)

# train the learner object with learning rate = 1e-2
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy
1,5.752823,4.926962,0.267100


In [ ]:
#Like a computer vision model, we can then unfreeze the model and fine-tune it.
learn.unfreeze()
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy
1,4.627915,3.901228,0.341061


In [ ]:
#Finally we save the encoder to be able to use it for classification in the next section.
learn.save_encoder('ft_enc')

In [ ]:
learn = text_classifier_learner(data_clas, drop_mult=0.5)
learn.load_encoder('ft_enc')
learn.fit_one_cycle(2, slice(1e-3,1e-2))

epoch,train_loss,valid_loss,accuracy
1,0.588080,0.522851,0.761695
2,0.567549,0.513127,0.767581


In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy
1,0.568293,0.481404,0.773262


In [ ]:
preds, targets = learn.get_preds()
predictions = np.argmax(preds, axis = 1)
pd.crosstab(predictions, targets,rownames=['Preds'], colnames=['targets'])

targets,0,1
Preds,,
0,12675,2359
1,2102,2402
